In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

#Pull in CSV paths
csvHL = "Resources/2007-2016-Homelessnewss-USA.csv"
csvRP = "Resources/price.csv"
csvpop = "Resources/datasets_1697_3211_Population-by-state (1).csv"
csvPPSQFT = "Resources/pricepersqft.csv"

In [2]:
#Prep CSV Paths for DataFrames
homeless_data = pd.read_csv(csvHL)
rentprice_data = pd.read_csv(csvRP)
population_data = pd.read_csv(csvpop)
pricepersqft_data = pd.read_csv(csvPPSQFT)

In [3]:
#Set DataFrames
homeless_df = pd.DataFrame(homeless_data)
rentprice_df = pd.DataFrame(rentprice_data)
population_df = pd.DataFrame(population_data)
pricepersqft_df = pd.DataFrame(pricepersqft_data)


In [4]:
#Transform Homeless CSV data to match Zillow Data Form

homelessfiltered_df = homeless_df.loc[homeless_df['Measures'] == "Total Homeless"]
#homelessfiltered_df[]
homeless_pdf = homelessfiltered_df.pivot_table(values='Count',
                                               index=homelessfiltered_df.State,
                                               columns='Year',
                                               aggfunc='first'
                                              )

#Reset Index and Rename columns
homeless_pdf.reset_index(inplace=True)
homeless_pdf.columns = ['State',
                        'HR2007',
                        'HR2008',
                        'HR2009',
                        'HR2010',
                        'HR2011',
                        'HR2012',
                        'HR2013',
                        'HR2014',
                        'HR2015',
                        'HR2016'
                       ]
homeless_pdf[["HR2007", "HR2008", "HR2009", "HR2010", "HR2011", "HR2012", "HR2013", "HR2014", "HR2015", "HR2016"]] = homeless_pdf[["HR2007", "HR2008", "HR2009", "HR2010", "HR2011", "HR2012", "HR2013", "HR2014", "HR2015", "HR2016"]].replace(',','', regex=True).astype(float)

#filter Homeless DataFrame to prep for Join with Rent Data, cleaned down to show variance and % CHG from 2011 to 2016
homeless_final = homeless_pdf.filter(['State', 'HR2011', 'HR2016'])
homeless_final["Var"] = homeless_final["HR2016"] - homeless_final["HR2011"]
homeless_final["% CHG"] = homeless_final["Var"] / homeless_final["HR2011"] * 100
homeless_final.head()


,State,HR2011,HR2016,Var,% CHG
0,AK,1223.0,1105.0,-118.0,-9.648406
1,AL,1950.0,1228.0,-722.0,-37.025641
2,AR,1276.0,808.0,-468.0,-36.677116
3,AZ,2047.0,2240.0,193.0,9.428432
4,CA,7067.0,6524.0,-543.0,-7.683600


In [5]:

# Perform Groupby on Rent Data by State with Average rent price
#rentprice_df.filter(['State', 'January 2011', 'January 2016'])
rentprice_mean = rentprice_df.groupby(['State']).mean()
rentprice_mean_years = rentprice_mean.filter(['State',
                                              'January 2011',
                                              'January 2012',
                                              'January 2013',
                                              'January 2014',
                                              'January 2015',
                                              'January 2016',
                                             ])

#Re-name columns to match format from homeless DF
rentprice_mean_years.columns = ['RP2011',
                                'RP2012',
                                'RP2013',
                                'RP2014',
                                'RP2015',
                                'RP2016'
                               ]
#rentprice_mean_years

rentprice_final = rentprice_mean_years.filter(['RP2011', 'RP2016'])
rentprice_final["RP Var"] = rentprice_final["RP2016"] - rentprice_final["RP2011"]
rentprice_final["RP % CHG"] = rentprice_final["RP Var"] / rentprice_final["RP2011"] * 100
rentprice_final.reset_index(inplace=True)
rentprice_final.head()

,State,RP2011,RP2016,RP Var,RP % CHG
0,AK,NaN,1644.000000,NaN,NaN
1,AL,959.351648,1023.484305,64.132657,6.685000
2,AR,828.773050,943.687075,114.914025,13.865560
3,AZ,1197.033613,1233.605042,36.571429,3.055171
4,CA,2019.433604,2322.005376,302.571772,14.983002


In [6]:
combined_df = homeless_final.set_index('State').join(rentprice_final.set_index('State'))
combined_clean = combined_df.dropna()
combined_clean.reset_index(inplace=True)
combined_final = combined_clean.style.format({
                             "RP2011":"${:20,.0f}",
                             "RP2016":"${:20,.0f}",
                             "RP Var":"${:20,.0f}",
                             "RP % CHG":"{:20,.1f}%",
                             "HR2011":"{:20,.0f}",
                             "HR2016":"{:20,.0f}",
                             "Var":"{:20,.0f}",
                             "% CHG":"{:20,.1f}%"
                            })

combined_final

,State,HR2011,HR2016,Var,% CHG,RP2011,RP2016,RP Var,RP % CHG
0,AL,"1,950","1,228",-722,-37.0%,$ 959,"$ 1,023",$ 64,6.7%
1,AR,"1,276",808,-468,-36.7%,$ 829,$ 944,$ 115,13.9%
2,AZ,"2,047","2,240",193,9.4%,"$ 1,197","$ 1,234",$ 37,3.1%
3,CA,"7,067","6,524",-543,-7.7%,"$ 2,019","$ 2,322",$ 303,15.0%
4,CO,"9,283","3,520","-5,763",-62.1%,"$ 1,398","$ 1,712",$ 315,22.5%
5,CT,"1,005",886,-119,-11.8%,"$ 1,733","$ 1,834",$ 101,5.9%
6,DC,"6,546","8,350","1,804",27.6%,"$ 2,241","$ 2,535",$ 294,13.1%
7,DE,"1,035","1,070",35,3.4%,"$ 1,329","$ 1,352",$ 23,1.7%
8,FL,"1,152","1,468",316,27.4%,"$ 1,400","$ 1,642",$ 242,17.3%
9,GA,"6,805","4,063","-2,742",-40.3%,"$ 1,116","$ 1,156",$ 39,3.5%


In [12]:
#html = combined_clean.to_html()
#print(html)


In [8]:
#df for Sam 
# homeless count of total homeless by state by years 2011 to 2016
#homeless_pdf
# Rent prices by state by years 2011 to 2016
#rentprice_mean_years

In [9]:
rentprice_all = rentprice_mean_years
homeless_all = homeless_pdf
rentprice_all.reset_index(inplace=True)

combined_all_data = homeless_all.set_index('State').join(rentprice_all.set_index('State'))

#rentprice_all

combined_all_data

,HR2007,HR2008,HR2009,HR2010,HR2011,HR2012,HR2013,HR2014,HR2015,HR2016,RP2011,RP2012,RP2013,RP2014,RP2015,RP2016
State,,,,,,,,,,,,,,,,
AK,974.0,1023.0,1267.0,1231.0,1223.0,1147.0,1122.0,1023.0,1208.0,1105.0,NaN,1507.666667,1519.166667,1595.583333,1568.000000,1644.000000
AL,2104.0,2104.0,2273.0,2273.0,1950.0,1707.0,1469.0,1329.0,1153.0,1228.0,959.351648,968.664835,959.551570,975.834081,1005.094170,1023.484305
AR,1822.0,1811.0,1425.0,1425.0,1276.0,1249.0,1066.0,1074.0,830.0,808.0,828.773050,856.673759,907.251701,927.312925,939.027211,943.687075
AZ,2997.0,2940.0,3236.0,3069.0,2047.0,2451.0,2435.0,2398.0,2402.0,2240.0,1197.033613,1178.033613,1167.621849,1175.823529,1200.117647,1233.605042
CA,7202.0,7202.0,7086.0,7086.0,7067.0,7053.0,7631.0,7567.0,6556.0,6524.0,2019.433604,1953.310298,1956.299731,2016.633065,2190.813172,2322.005376
CO,4450.0,5188.0,5267.0,5267.0,9283.0,9283.0,2267.0,2188.0,3545.0,3520.0,1397.872611,1423.974522,1446.089888,1494.853933,1600.837079,1712.449438
CT,919.0,846.0,825.0,944.0,1005.0,902.0,1059.0,1083.0,913.0,886.0,1732.548571,1739.525714,1722.914286,1751.468571,1785.834286,1833.931429
DC,5320.0,6044.0,6228.0,6539.0,6546.0,6954.0,6865.0,7748.0,7298.0,8350.0,2241.000000,2335.000000,2411.000000,2482.000000,2433.000000,2535.000000
DE,1061.0,933.0,1130.0,982.0,1035.0,1008.0,946.0,901.0,953.0,1070.0,1328.921053,1273.315789,1254.578947,1287.578947,1331.578947,1351.842105


In [10]:
rentprice_mean_years

,State,RP2011,RP2012,RP2013,RP2014,RP2015,RP2016
0,AK,NaN,1507.666667,1519.166667,1595.583333,1568.000000,1644.000000
1,AL,959.351648,968.664835,959.551570,975.834081,1005.094170,1023.484305
2,AR,828.773050,856.673759,907.251701,927.312925,939.027211,943.687075
3,AZ,1197.033613,1178.033613,1167.621849,1175.823529,1200.117647,1233.605042
4,CA,2019.433604,1953.310298,1956.299731,2016.633065,2190.813172,2322.005376
5,CO,1397.872611,1423.974522,1446.089888,1494.853933,1600.837079,1712.449438
6,CT,1732.548571,1739.525714,1722.914286,1751.468571,1785.834286,1833.931429
7,DC,2241.000000,2335.000000,2411.000000,2482.000000,2433.000000,2535.000000
8,DE,1328.921053,1273.315789,1254.578947,1287.578947,1331.578947,1351.842105
9,FL,1400.312734,1368.091760,1419.675325,1504.649351,1567.129870,1642.417440


In [15]:
engine = create_engine('sqlite:///Transform-Tim.db', echo=True)
sqlite_connect = engine.connect()
Base = automap_base()
Base.prepare(engine, reflect = True)
Base.classes.keys()

2020-08-26 18:13:37,388 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-26 18:13:37,388 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 18:13:37,389 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-26 18:13:37,390 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 18:13:37,391 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-08-26 18:13:37,392 INFO sqlalchemy.engine.base.Engine ()


[]

In [16]:
sqlite_table = "Rent Prices Vs Homelessness"

In [17]:
combined_all_data.to_sql(sqlite_table, sqlite_connect, if_exists='fail')

2020-08-26 18:19:06,125 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Rent Prices Vs Homelessness")
2020-08-26 18:19:06,126 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 18:19:06,127 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Rent Prices Vs Homelessness")
2020-08-26 18:19:06,128 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 18:19:06,131 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Rent Prices Vs Homelessness" (
	"State" TEXT, 
	"HR2007" FLOAT, 
	"HR2008" FLOAT, 
	"HR2009" FLOAT, 
	"HR2010" FLOAT, 
	"HR2011" FLOAT, 
	"HR2012" FLOAT, 
	"HR2013" FLOAT, 
	"HR2014" FLOAT, 
	"HR2015" FLOAT, 
	"HR2016" FLOAT, 
	"RP2011" FLOAT, 
	"RP2012" FLOAT, 
	"RP2013" FLOAT, 
	"RP2014" FLOAT, 
	"RP2015" FLOAT, 
	"RP2016" FLOAT
)


2020-08-26 18:19:06,131 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 18:19:06,133 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 18:19:06,134 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Rent Prices Vs Homelessness_State" 

In [23]:
test_df = pd.read_sql_table('Rent Prices Vs Homelessness', 'sqlite:///Transform-Tim.db')

In [24]:
test_df

,State,HR2007,HR2008,HR2009,HR2010,HR2011,HR2012,HR2013,HR2014,HR2015,HR2016,RP2011,RP2012,RP2013,RP2014,RP2015,RP2016
0,AK,974.0,1023.0,1267.0,1231.0,1223.0,1147.0,1122.0,1023.0,1208.0,1105.0,NaN,1507.666667,1519.166667,1595.583333,1568.000000,1644.000000
1,AL,2104.0,2104.0,2273.0,2273.0,1950.0,1707.0,1469.0,1329.0,1153.0,1228.0,959.351648,968.664835,959.551570,975.834081,1005.094170,1023.484305
2,AR,1822.0,1811.0,1425.0,1425.0,1276.0,1249.0,1066.0,1074.0,830.0,808.0,828.773050,856.673759,907.251701,927.312925,939.027211,943.687075
3,AZ,2997.0,2940.0,3236.0,3069.0,2047.0,2451.0,2435.0,2398.0,2402.0,2240.0,1197.033613,1178.033613,1167.621849,1175.823529,1200.117647,1233.605042
4,CA,7202.0,7202.0,7086.0,7086.0,7067.0,7053.0,7631.0,7567.0,6556.0,6524.0,2019.433604,1953.310298,1956.299731,2016.633065,2190.813172,2322.005376
5,CO,4450.0,5188.0,5267.0,5267.0,9283.0,9283.0,2267.0,2188.0,3545.0,3520.0,1397.872611,1423.974522,1446.089888,1494.853933,1600.837079,1712.449438
6,CT,919.0,846.0,825.0,944.0,1005.0,902.0,1059.0,1083.0,913.0,886.0,1732.548571,1739.525714,1722.914286,1751.468571,1785.834286,1833.931429
7,DC,5320.0,6044.0,6228.0,6539.0,6546.0,6954.0,6865.0,7748.0,7298.0,8350.0,2241.000000,2335.000000,2411.000000,2482.000000,2433.000000,2535.000000
8,DE,1061.0,933.0,1130.0,982.0,1035.0,1008.0,946.0,901.0,953.0,1070.0,1328.921053,1273.315789,1254.578947,1287.578947,1331.578947,1351.842105
9,FL,1012.0,1361.0,1999.0,1999.0,1152.0,921.0,1049.0,1377.0,1198.0,1468.0,1400.312734,1368.091760,1419.675325,1504.649351,1567.129870,1642.417440
